In [17]:
####################################################################################
# This test code shows how new_basin_cities looks like
# basin at each city
distance=100
min_of_cities=0
max_of_basinnum=1000
target_city = 14
####################################################################################

h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08'
basin_cities_path = f'{h08dir}/global_city/dat/riv_num_/basin_to_cities_{distance}km.json'

with open(basin_cities_path, 'r') as json_file:
    tmp = json.load(json_file)
    
sorted_keys = sorted(tmp.keys(), key=lambda k: float(k))

target_key_list = []

for key in sorted_keys:
    value = tmp[key]
    if min_of_cities < len(value) and float(key) < max_of_basinnum:
        for i in value:
            if int(float(i)) == target_city:
                target_key_list.append(key)
                
print(f'target_city: {target_city}')
print(f'target_key_list: {target_key_list}')

target_city: 14
target_key_list: ['23.0']


In [18]:
# This test code shows how cities in target_basin looks like

target_key = '23.0'
value = tmp[target_key]
print(f'target_key: {target_key}')
print(f'value: {value}')

target_key: 23.0
value: ['14', '33', '111', '146', '221', '260', '272', '276', '284', '296', '354', '370', '466', '536', '578', '715', '764', '813', '849', '1091', '1103', '1196', '1220', '1295', '1405', '1430', '1519', '1535', '1536', '1563', '1582', '1636', '1689']


In [19]:
# This test code shows riv_num index of tmp dictionary

keys_lst = list(tmp.keys())
key_index = keys_lst.index(target_key)
print(f'rivnum_key_index: {key_index}')

rivnum_key_index: 15


In [20]:
# This test code shows ava & mpcy

import pandas as pd

#mpcy_int_path = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08/global_city/dat/cty_lst_/gpw4/mcy_pop_water_12region.txt'
#column_names = ['index', 'flag', 'rate', 'wup', 'pop', 'ava', 'mpcy', 'potential', 'region', 'country', 'cityname', '12region']
#df = pd.read_csv(mpcy_int_path, delimiter='|', header=None, names=column_names)

mpcy_int_path = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08/wsi/dat/availablewater/updown_availability_100km_abandon_revised.txt'
df = pd.read_csv(mpcy_int_path, delimiter='|', header=0)
df_filtered_test = df[df['index'] == target_city]
display(df_filtered_test.head())

,index,flag,rate,wup,pop,ava,mpcy,potential,region,country,cityname,12region,ava_int,mpcy_int
13,14,False,0.92072,12611.924,11612047.0,4.488725e+08,38.655756,37.0,southern asia,Pakistan,Karachi,South Asia,8.267551e+08,71.19805


In [26]:
import numpy as np

def discharge_withincity(city_num, riv_num):
    print(f'city_num: {city_num}, riv_num: {riv_num}')
    distance = 100
    h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08'
    rivout_path = f'{h08dir}/wsi/dat/riv_out_/W5E5LR__00000000.gl5'
    rivnum_path = f'{h08dir}/global_city/dat/riv_num_/rivnum.CAMA.gl5'
    rivout_gl5 = np.fromfile(rivout_path, dtype='float32').reshape(2160, 4320)
    rivnum_gl5 = np.fromfile(rivnum_path, dtype='float32').reshape(2160, 4320)
    prf_dir = f'{h08dir}/global_city/dat/cty_prf_'
    int_dir = f'{h08dir}/global_city/dat/cty_int_/{distance}km_samebasin'
    
    prf_path = f'{prf_dir}/vld_cty_/city_{city_num:08}.gl5'
    int_path = f'{int_dir}/city_{city_num:08}.gl5'
    
    prf = np.fromfile(prf_path, dtype='float32').reshape(2160, 4320)
    intake = np.fromfile(int_path, dtype='float32').reshape(2160, 4320)
    
    dis_pprf = np.sum(rivout_gl5[(prf==1) & (rivnum_gl5==riv_num)])* 60 * 60 * 24 * 365 / (1000) # m3/year
    dis_sprf = np.sum(rivout_gl5[(prf==1) & (rivnum_gl5!=riv_num)])* 60 * 60 * 24 * 365 / (1000) # m3/year
    dis_pint = np.sum(rivout_gl5[(intake==1) & (rivnum_gl5==riv_num)])* 60 * 60 * 24 * 365 / (1000) # m3/year
    dis_sint = np.sum(rivout_gl5[(intake==1) & (rivnum_gl5!=riv_num)])* 60 * 60 * 24 * 365 / (1000) # m3/year

    print(f'dis_pprf: {dis_pprf/1e9}')
    print(f'dis_sprf: {dis_sprf/1e9}')
    print(f'dis_pint: {dis_pint/1e9}')
    print(f'dis_sint: {dis_sint/1e9}')

    prf_total = dis_pprf + dis_sprf
    int_total = dis_pint + dis_sint

    print(f'prf_total: {prf_total/1e9}')
    print(f'int_total: {int_total/1e9}')

discharge_withincity(target_city, target_key)

city_num: 14, riv_num: 23.0
dis_pprf: 0.0
dis_sprf: 0.4488724603125
dis_pint: 0.0
dis_sint: 0.37788264534375
prf_total: 0.4488724603125
int_total: 0.37788264534375
